# 1. Setup Azure
In this notebook, we will set up the project in Azure by creating the resources we need.

We start by creating the resource group that we'll put all resources into. Then we create our storage account which will be used to store all data (include logs). Finally we set up our Service Bus namespace, and a queue attached to it.

---

### Import packages and load .env

In [ ]:
import re
from dotenv import set_key, get_key, find_dotenv
from pathlib import Path
%load_ext dotenv

In [ ]:
env_path = find_dotenv()
if env_path=='':
    Path('.env').touch()
    env_path = find_dotenv()

### Define variables

Set variables for the project:

- `subscription_id` - the subscription id for your Azure account. Use `az account list -o table` to list all subscriptions
- `resource_group` - the name for the resource group you'll be using for this project. You can think of resource groups as logical containers for the resources you'll create in this tutorial.
- `region` - the region you wish to deploy your resources in:
    - You can see a list of the regions under the key 'name' when running the command `az account list-locations`. 
    - Not all regions support GPU enabled VMs. You can check [here](https://azure.microsoft.com/en-us/pricing/details/virtual-machines/linux/)

In [ ]:
subscription_id = "<your-subscription-id>"
resource_group = "<your-resource-group>"
region = "<your-region>"

Set your storage account variables

- `storage_account_name` - the name of your storage account
- `storage_container_name` - the container in the storage account is a logical container for individual blobs. For this project, we'll store all content into a single container for ease-of-use. Set your container name.
- `model_dir` - the name of directory you wish to store your models

In [ ]:
storage_account_name = "<your-storage-account-name>"
storage_container_name = "aks"
model_dir = "models"

Set the Service Bus variables:

- `namespace` - the namespace for your Service Bus - this can simply be thought of as a logical container for Service Bus
- `queue` - the name of the queue. The queue belongs to the namespace.

In [ ]:
namespace = "<your-namespace>"
queue = "<your-queue>"

### Set up your resource group
Make sure you've identified which subscription_id and region to use. Create a new resource group to contain all the resources that we create.

This section of the notebook will walk through setting up the resource group using the __az cli__.

In [ ]:
!az account set -s $subscription_id

Create the resource group which we'll put our storage account and all other resources for this project into.

In [ ]:
!az group create -l $region -n $resource_group

### Create Azure blob storage
In this section of the notebook, we'll create an Azure blob storage that we'll use throughout the tutorial. This object store will be used to store input and output images as well as any supplementary data such as logs and other scripts that will be used in this workflow.

Use the __az cli__ to create the account

In [ ]:
!az storage account create -n $storage_account_name -g $resource_group --query 'provisioningState'

Use the __az cli__ to grab the keys of the storage account that was just created. The `--quote '[0].value'` part of the command simply means to select the _value_ of the _zero-th indexed_ of the set of keys.

In [ ]:
key = !az storage account keys list --account-name $storage_account_name -g $resource_group --query '[0].value'

The stdout from the command above is stored in a string array of 1. Select the element in the array and ttrip opening and closing quotation marks.

In [ ]:
storage_account_key = str(key[0][1:-1]) # this is used to strip opening and closing quotation marks

Use the __az cli__ to create the container in the storage account

In [ ]:
!az storage container create \
    --account-name $storage_account_name \
    --account-key $storage_account_key \
    --name $storage_container_name

### Add Models to storage

When you download this repo, the project comes with a folder `models`. In the folder, there are 4 pre-built model files. We want to copy this entire folder up to storage so that it can be used by aks later on in this project.

In [ ]:
!azcopy \
    --source models \
    --destination "https://"$storage_account_name".blob.core.windows.net/"$storage_container_name"/"$model_dir \
    --dest-key $storage_account_key \
    --recursive \
    --resume "."

### Create Service Bus

Create the namespace.

In [ ]:
!az servicebus namespace create \
    --resource-group $resource_group \
    --name $namespace 

Create a service bus queue. Set the lock duration to 5 minutes. This means that the lock for each queue message will last for 5 minutes.

In [ ]:
!az servicebus queue create \
    --resource-group $resource_group \
    --namespace-name $namespace \
    --name $queue \
    --lock-duration PT5M

Now that we've created our Service Bus namespace and queue, we need to get the key-name/key-value pair so that we can access it.

By default, your Service Bus resource will come with an key-value "authorization rule" pair - its key name will have the value: "RootManageSharedAccessKey". The following command will get the key name for the "authorization rule", and assign it as `sb_key_name`.

In [ ]:
sb_key_name = !az servicebus namespace authorization-rule list \
    --resource-group $resource_group \
    --namespace-name $namespace \
    -o json --query "[0].name"

sb_key_name = str(sb_key_name[0][1:-1])

Get the primary key value to "RootManageSharedAccessKey".

In [ ]:
sb_credentials = !az servicebus namespace authorization-rule keys list \
    --resource-group $resource_group \
    --namespace-name $namespace \
    --name $sb_key_name \
    -o json --query "primaryKey"

In [ ]:
sb_key_value = re.findall(r'"(.*?)"', str(sb_credentials))[0]

---

### Conclusion

Since we'll be using these settings throughout this tutorial, we'll also same them to the `.env` file.

In [ ]:
set_key(env_path, "SUBSCRIPTION_ID", subscription_id)
set_key(env_path, "RESOURCE_GROUP", resource_group)
set_key(env_path, "REGION", region)
set_key(env_path, "STORAGE_ACCOUNT_NAME", storage_account_name)
set_key(env_path, "STORAGE_ACCOUNT_KEY", storage_account_key)
set_key(env_path, "STORAGE_CONTAINER_NAME", storage_container_name)
set_key(env_path, "STORAGE_MODEL_DIR", model_dir)
set_key(env_path, "SB_SHARED_ACCESS_KEY_NAME", sb_key_name)
set_key(env_path, "SB_SHARED_ACCESS_KEY_VALUE", sb_key_value)
set_key(env_path, "SB_NAMESPACE", namespace)
set_key(env_path, "SB_QUEUE", queue)

Check that our `.env` file looks correct.

In [ ]:
!cat .env

Continue to the next [notebook](/notebooks/02_style_transfer_locally.ipynb).